# L4: 묘사 & 생성 게임 🖍️

당신의 HuggingFace API 키, 그리고 관련된 라이브러리를 불러오세요

In [ ]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # 실제 로컬의 .env 파일 읽기
hf_api_key = os.environ['HF_API_KEY']

In [ ]:
# Helper 함수
import requests, json

# 여러 개의 엔드포인트를 호출합니다!
def get_completion(inputs, parameters=None, ENDPOINT_URL=""):
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }   
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL,
                                headers=headers,
                                data=json.dumps(data))
    return json.loads(response.content.decode("utf-8"))

In [ ]:
# text-to-image (텍스트 -> 이미지)
TTI_ENDPOINT = os.environ['HF_API_TTI_BASE']
# image-to-text (이미지 -> 텍스트)
ITT_ENDPOINT = os.environ['HF_API_ITT_BASE']

## `gr.Blocks()`로 게임 만들기

In [ ]:
# 3,4번 레슨에 사용된 함수를 불러옵니다
# 이미지 -> base64 형식으로 변환
def image_to_base64_str(pil_image):
    byte_arr = io.BytesIO()
    pil_image.save(byte_arr, format='PNG')
    byte_arr = byte_arr.getvalue()
    return str(base64.b64encode(byte_arr).decode('utf-8'))

# base64 -> pil 형식으로 변환
def base64_to_pil(img_base64):
    base64_decoded = base64.b64decode(img_base64)
    byte_stream = io.BytesIO(base64_decoded)
    pil_image = Image.open(byte_stream)
    return pil_image

# 이미지에 캡션 달기
def captioner(image):
    base64_image = image_to_base64_str(image)
    result = get_completion(base64_image, None, ITT_ENDPOINT)
    return result[0]['generated_text']

# 텍스트로 이미지 생성하기
def generate(prompt):
    output = get_completion(prompt, None, TTI_ENDPOINT)
    result_image = base64_to_pil(output)
    return result_image

### 우선 이미지에 대한 캡션을 생성합니다

In [ ]:
import gradio as gr 
with gr.Blocks() as demo:
    gr.Markdown("# Describe-and-Generate game 🖍️")
    image_upload = gr.Image(label="Your first image",type="pil") # 이미지 입력 받기
    btn_caption = gr.Button("Generate caption") # 생성 버튼
    caption = gr.Textbox(label="Generated caption")
    
    btn_caption.click(fn=captioner, inputs=[image_upload], outputs=[caption])

gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT1']))

### 이제 텍스트로부터 이미지 생성을 추가하죠

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Describe-and-Generate game 🖍️")
    image_upload = gr.Image(label="Your first image",type="pil")
    btn_caption = gr.Button("Generate caption")
    caption = gr.Textbox(label="Generated caption")
    btn_image = gr.Button("Generate image")
    image_output = gr.Image(label="Generated Image")
    btn_caption.click(fn=captioner, inputs=[image_upload], outputs=[caption]) # 캡션(텍스트)이 output
    btn_image.click(fn=generate, inputs=[caption], outputs=[image_output]) # 위에서 나온 output을 input으로 사용

gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT2'])) # 최종 output은 이미지

### 이젠 두 단계로 나누지 않고 한 단계에 진행합니다

In [ ]:
def caption_and_generate(image):
    caption = captioner(image) # 이미지로부터 캡션 생성
    image = generate(caption) # 캡션으로부터 이미지 생성
    return [caption, image]

with gr.Blocks() as demo:
    gr.Markdown("# Describe-and-Generate game 🖍️")
    image_upload = gr.Image(label="Your first image",type="pil")
    btn_all = gr.Button("Caption and generate")
    caption = gr.Textbox(label="Generated caption")
    image_output = gr.Image(label="Generated Image")

    btn_all.click(fn=caption_and_generate, inputs=[image_upload], outputs=[caption, image_output]) # 두 개의 output을 반환

gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT3']))

In [ ]:
gr.close_all()